In [ ]:
try:
    import piplite
    await piplite.install(['ipywidgets==7.7.2', 'ipyflex', 'ipyvuetify==1.8.2', 'ipyvue==1.7.0', 'plotly', 'pandas'])
except:
    pass

In [ ]:
import os
from datetime import datetime
import ipyvuetify as v
import ipywidgets as ipw
from ipyflex import FlexLayout
import pandas
import json

In [ ]:
class TickerData:
    def __init__(self, ticker: str) -> None:
        ticker = ticker.upper()
        self.fetch_ticker(ticker)

    def fetch_ticker(self, ticker: str):
        with open('data.json', 'r') as f:
            cache = json.load(f)
        self._cached_data = cache.get(ticker, None)
        if self._cached_data:
            self._info = self._cached_data['data']['info']
            self.news = self._cached_data['data']['news']
            self._price = pandas.DataFrame.from_dict(self._cached_data['price'])
            # self._price.index = [datetime.fromtimestamp(int(x)/1000) for x in self._price.index]
            self._balance_sheet = pandas.DataFrame.from_dict(self._cached_data['balance_sheet'])
            self._analysis = pandas.DataFrame.from_dict(self._cached_data['analysis'])
    @property
    def financial_info(self):
        data = [
            {'title': 'Market cap', 'value': self._info['marketCap']},
            {'title': 'PE Ratio', 'value': self._info['forwardPE']},
            {'title': 'Total revenue', 'value': self._info['totalRevenue']},
            {'title': 'Gross profit', 'value': self._info['grossProfits']},
            {'title': 'Debt to equity', 'value': self._info['debtToEquity']},
            {'title': 'Profit margin', 'value': self._info['profitMargins']},
        ]
        return data

    def price(self):
        return self._price

    @property
    def balance_sheet(self):
        return self._balance_sheet

    @property
    def analysis(self):
        return self._analysis.transpose()


In [ ]:

from typing import Dict, List
import json
import ipyvuetify as v
import ipywidgets as ipw
import plotly.graph_objects as go
from plotly.subplots import make_subplots

v.theme.dark = True


def new_factory(news: List[Dict]) -> v.Html:
    children = []
    for new in news:
        date = datetime.fromtimestamp(new['providerPublishTime'])
        btn = v.Btn(
            small=True,
            text=True,
            block=True,
            children=['Open'],
            href=new['link'],
            target='_blank',
        )
        card = v.Card(
            outlined=True,
            children=[
                v.CardTitle(
                    children=[new['title']],
                    style_='font-size: 1.1rem',
                ),
                v.CardSubtitle(
                    children=[
                        f'{new["publisher"]} ({date.strftime("%m/%d/%Y, %H:%M")})'
                    ]
                ),
                v.CardActions(children=[btn]),
            ],
        )
        children.append(card)
    return v.Html(tag='div', class_='d-flex flex-column', children=children)


def financial_info_factory(data: List[Dict], logo_url: str = None) -> v.Html:
    children = []
    if logo_url is not None:
        logo = v.Card(
            outlined=True,
            class_='ma-1',
            children=[v.Img(src=logo_url, height='100px', contain=True)],
            style_='width: calc(14.28% - 8px); min-width: 150px',
        )
        children.append(logo)

    for item in data:
        card = v.Card(
            outlined=True,
            class_='ma-1',
            children=[
                v.CardTitle(
                    primary_title=True,
                    children=[item['title']],
                    style_='font-size: 18px; color: #51ef98',
                ),
                v.CardText(children=[str(item['value'])]),
            ],
            style_='width: calc(14.28% - 8px); min-width: 150px',
        )
        children.append(card)
    return v.Html(
        tag='div',
        class_='d-flex flex-row',
        children=children,
        style_='flex-wrap: wrap',
    )


def price_chart_factory(df: List, ticker: str = '') -> ipw.Widget:
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{'secondary_y': True}]])
    index = [datetime.fromtimestamp(int(x)/1000).strftime('%m/%d') for x in df.index]
    # include candlestick with rangeselector
    fig.add_trace(
        go.Candlestick(
            x=index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='OHLC',
        ),
        secondary_y=True,
    )
    fig.add_trace(
        go.Bar(
            x=index,
            y=df['Volume'],
            marker_color='rgba(91, 91, 91, 0.73)',
            name='Volume',
        ),
        secondary_y=False,
    )

    fig.layout.yaxis2.showgrid = False
    fig.update_layout(
        autosize=True,
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        title={
            'text': f'{ticker.upper()} PRICE CHART',
            'xanchor': 'center',
            'yanchor': 'top',
            'x': 0.5,
        },
    )
    widget = go.FigureWidget(fig, layout=ipw.Layout(height='100%'))
    return widget


def price_history_factory(df: List, ticker: str = '') -> ipw.Widget:
    # include candlestick with rangeselector
    index = [datetime.fromtimestamp(int(x)/1000).strftime('%m/%d') for x in df.index]
    widget = go.FigureWidget(go.Scatter(x=index, y=df['Close']))
    widget.update_layout(
        autosize=True,
        template='plotly_dark',
        title={
            'text': f'{ticker.upper()} PRICE HISTORY',
            'xanchor': 'center',
            'yanchor': 'top',
            'x': 0.5,
        },
    )
    return widget


def text_widget(title: str, text: str) -> ipw.Widget:
    return v.Card(
        outlined=True,
        children=[
            v.CardTitle(
                children=[title],
                style_='font-size: 1.1rem',
            ),
            v.CardText(children=[text]),
        ],
    )


def balance_sheet_factory(df) -> ipw.Widget:
    items = []
    for i in range(df.shape[0]):
        row = df.iloc[i]
        item = {'name': row.name}
        item.update(json.loads(row.to_json()))
        items.append(item)
    titles = [x for x in items[0].keys() if x != 'name']
    headers = [
        {
            'text': 'Property',
            'align': 'start',
            'sortable': False,
            'value': 'name',
        }
    ]
    for title in titles:
        date = datetime.fromtimestamp(int(title) / 1000)
        header = {'text': date.strftime('%m/%d/%Y, %H:%M'), 'value': title}
        headers.append(header)
    return v.DataTable(
        headers=headers,
        items=items,
    )


def analysis_factory(df) -> ipw.Widget:
    items = []
    for i in range(df.shape[0]):
        row = df.iloc[i]
        item = {'name': row.name}
        item.update(json.loads(row.to_json()))
        items.append(item)
    titles = [x for x in items[0].keys() if x != 'name']
    headers = [
        {
            'text': 'Property',
            'align': 'start',
            'sortable': False,
            'value': 'name',
        }
    ]
    for title in titles:
        header = {'text': title, 'value': title}
        headers.append(header)
    return v.DataTable(
        headers=headers,
        items=items,
    )


In [ ]:
editable = False
height = "900px"

In [ ]:
ticker_name = "NVDA"
ticker = TickerData(ticker_name)
news = new_factory(ticker.news)
info = financial_info_factory(ticker.financial_info, ticker._info["logo_url"])
price = price_chart_factory(ticker.price(), ticker_name)
history = price_history_factory(ticker.price(), ticker_name)
summary = text_widget("Business Summary ", ticker._info["longBusinessSummary"])
balance_sheet = balance_sheet_factory(ticker.balance_sheet)
analysis = analysis_factory(ticker.analysis)
widgets = {
    "news": news,
    "info": info,
    "price": price,
    "history": history,
    "summary": summary,
    "balance_sheet": balance_sheet,
    "analysis": analysis
}

In [ ]:
w = FlexLayout(
    widgets,
    style={"height": height},
    header={"title": "STOCK DASHBOAD", "style": {"backgroundColor": "rgb(53 53 53)"}, "buttons":["export","import"]},
    template="ticker2.json",
    editable=False,
)
style = ipw.HTML("""<style>.js-plotly-plot {height: 100%;}</style> """)

In [ ]:
w